# MARCH MADNESS PREDICTOR

Authors: Connor Finn, Riley Greene <br>
Date: 1/24/20 <br>
Warren Buffet is still paying 1 billion for a perfect bracket

In [2]:
# our imports for the model
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import lxml.html as lh

# a few constants
start_year = 2010

Because basketball has changed so much over the past years, we believe that data prior to the year 2010 could be detrimental to our model. For this reason, the next block of code will be used to slim the data down from the 1985 - 2018 seasons to a dataframe including the the 2020 - 2018 NCAA tournaments.

In [3]:
# read ncaa tournament results into dataframe ( data obtained from Kaggle)
ncaa_total = pd.read_csv('NCAATourneyCompactResults.csv')  
# read team ID dataframe (from Kaggle)
team_data = pd.read_csv('teams.csv')  
# get the first index for the desired year
start = min(ncaa_total[ncaa_total.iloc[:,0] == start_year].index.values.astype(int)) 
# shorten our dataframe
ncaa_short = ncaa_total.iloc[start: , :]

The team_id provides names written as 'Penn State'. The data we will be collecting is from Sports-reference.com. The url for penn state's data is 'https://www.sports-reference.com/cbb/schools/penn-state/2020.html' <br>
1) the name needs to be adjusted to no spaces, all lower caps <br>
2) the overall framework is 'https://www.sports-reference.com/cbb/schools/NAME/YEAR.html'<br>

In [9]:
"""
* this function will be used to transform a provided name to one that can be input into a sports-reference url
* this will likely need to be adjusted as new teams are included
* THIS NEEDS A UNIT TEST (i.e. run all our team names through once to see if they pass)
"""
def clean_team_name(name):
    # name: string
    team = name
    team = team.lower()
    team = team.replace(' ' , '-')
    team = team.replace('(' , '')
    team = team.replace(')' , '')
    team = team.replace('.' , '')
    team = team.replace( "'" , '')
    team = team.replace("&" , "")
    team = team.replace("university-of-" , "") 
    return team
def test_team_names(team_names):
    # team_names: list of strings 
    errors = 0
    for name in team_names:
        url_test =  "https://www.sports-reference.com/cbb/schools/" + name + "/"
        try:
            pageWL = requests.get(url_test)
        except:
            print(name , " did not return a valid search criteria")
            errors += 1
    print("finished unit test. There were " , errors , " errors.")
    return errors

team_data is a dataframe which has the columns: 'TeamID', 'TeamName', 'FirstD1Season', 'LastD1Season'.  We want to create a new column which has the team names in accordance to the sports-reference.com framework. 

In [10]:
# get a list of all the team names in our dataframe
team_names = team_data.TeamName.values.tolist()
# clean every name in the team_data dataframe
sr_names = []
for name in team_names:
    sr_names += [clean_team_name(name)]
# run the test: if no error messages show up, then we add the list to dataframe
num_errors = test_team_names(sr_names)
if num_errors ==0:
    team_data["SrNames"] = sr_names

finished unit test. There were  0  errors.


In [ ]:
Using the two above dataframes, generate a list of team's that competed in the 2010-2018 NCAA tournaments. From there we can collect season data for each team from sports-reference.com and start the ML model

In [ ]:
# get the number of rows in the NCAA tournament dataframes ( number of games)
ncaa_rows = ncaa_short.shape[0]
# get all unique team id's for competitors in the tournaments
all_team_id = ncaa_short.WTeamID.append(ncaa_short.LTeamID).unique()
# Can below be done more efficienty?
team_names = []
for id1 in all_team_id:
    teamIndex = team_data[team_data.TeamID==id1].index.values.astype(int)[0]  # this line could be slicker
    teamName = team_data.TeamName.iloc[teamIndex]
    team_names += [teamName]


'team_names' is now have a compiled list of team names that have competed in the 2010 - 2018 NCAA tournaments. Next, we would like to collect the season data for these teams.